In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os

In [11]:
# 迁移学习： 从ImageNet中已经训练好的神经网络 迁移并适配
data_dir = '06_transfer_ant_bee_data'
image_size = 224

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                     transforms.Compose([
                                         transforms.RandomResizedCrop(image_size), #  裁剪到指定大小
                                         transforms.RandomHorizontalFlip(), # 默认0.5的概率随机水平翻转
                                         transforms.ToTensor(), # RBG三个通道数据 归一化到[0, 1](除以255)
                                         
                                         # 简单来说就是将数据按通道进行计算，将每一个通道的数据先计算出其方差与均值，
                                         # 然后再将其每一个通道内的每一个数据减去均值，再除以方差，x = (x - mean) / std，得到归一化后的结果。
                                         # 在深度学习图像处理中，标准化处理之后，可以使数据更好的响应激活函数，提高数据的表现力，
                                         # 减少梯度爆炸和梯度消失的出现。
                                         
                                         # ImageNet数据集的均值和方差为：mean=(0.485, 0.456, 0.406)，std=(0.229, 0.224, 0.225)，
                                         # 因为这是在百万张图像上计算而得的，所以我们通常见到在训练过程中使用它们做标准化。
                                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 归一化结果不一定是[-1，1]
                                     ])
                                    )
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'),
                                   transforms.Compose([
                                       transforms.Resize(256),  # 图像放大到 256 * 256
                                       transforms.CenterCrop(image_size), # 从中心区域切割 224 * 224
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                   ])
                                  )


# 创建数据加载器
# num_workers 指定工作进程的个数，负责加载batch，一般设置为cpu核心数或一半
# batch_size: 每批的数据大小
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 4, shuffle = True, num_workers = 4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 4, shuffle = True, num_workers = 4)

# dataset.classes: 读取数据集下文件夹名并将其作为类别存入列表，如 train目录下有 ants和 bees两个文件夹，所以类别数为2
num_classes = len(train_dataset.classes)                                         